In [ ]:
%pip install spacy_transformers

In [ ]:
%pip install -U spacy

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [3]:
spacy.__version__

'3.6.1'

In [4]:
# Load the annotated data from a JSON file
import json
cv_data = json.load(open(r"./data/training/train_data.json"))

In [5]:
len(cv_data)

200

In [6]:
cv_data[1]

["Harini Komaravelli Test Analyst at Oracle, Hyderabad  Hyderabad, Telangana - Email me on Indeed: indeed.com/r/Harini- Komaravelli/2659eee82e435d1b  ➢ 6 Yrs. of IT Experience in Manual and Automation testing.  WORK EXPERIENCE  QA Analyst  Oracle  Test Analyst at Oracle, Hyderabad  Infosys Ltd -  Hyderabad, Telangana -  November 2011 to February 2016  Hyderabad from Nov 2011 to Feb17 2016 ➢ Worked in Tata Consultancy Services, Hyderabad from Feb 24 to Apr 11 2017 ➢ Currently working as a Test Analyst at Oracle, Hyderabad  QA Analyst with 6 years of IT experience  Oracle  EDUCATION  MCA  Osmania University  B.Sc. in Computer Science  Osmania University  SKILLS  Functional Testing, Blue Prism, Qtp  ADDITIONAL INFORMATION  Area of Expertise:  ➢ Familiar with Agile Methodologies. ➢ Having knowledge in Energy (Petroleum) & Health Care domains. ➢ Involved in preparation of Test Scenarios. ➢ Preparing Test Data for the test cases.  https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?

In [7]:
!python -m spacy init fill-config data/training/base_config.cfg /data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
\data\training\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True   # if entities are overlapping
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
        doc.ents = ents
        db.add(doc)
    except:
        pass

  return db


In [ ]:
%pip install scikit-learn

In [9]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [10]:
# Display the number of items in the training and testing sets
len(train), len(test)

(140, 60)

In [11]:
# Open a file to log errors during annotation processing
file = open('./data/model_data/error_file.txt','w', encoding='utf-8')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('./data/model_data/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('./data/model_data/test_data.spacy')

# Close the error log file
file.close()

100%|██████████| 60/60 [00:01<00:00, 57.25it/s]


In [12]:
len(db.tokens)

60

In [27]:
%pip install cupy

  Using cached cupy-12.2.0.tar.gz (2.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [67 lines of output]
      Clearing directory: C:\Users\domin\AppData\Local\Temp\pip-install-w0y5zh7a\cupy_e64f83818c1744389c1f4f4c033cd689\cupy\.data
      Looking for NVTX: C:\Program Files\NVIDIA Corporation\Nsight Systems *\target-windows-x64\nvtx
      NVTX could not be found
      
      -------- Configuring Module: cuda --------
      Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      **************************************************
      *** WARNING: Cannot check compute capability
      Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      **************************************************
      
      ******************************************

In [29]:
%pip install cupy==<desired_version>

Note: you may need to restart the kernel to use updated packages.


The syntax of the command is incorrect.


In [13]:
!python -m spacy train data/training/config.cfg --output data/model_output --paths.train data/model_data/train_data.spacy --paths.dev data/model_data/test_data.spacy --gpu-id 0

ℹ Saving to output directory: data\model_output
ℹ Using GPU: 0


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\domin\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "c:\Users\domin\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\cli\_util.py", line 92, in setup_cli
    command(prog_name=COMMAND)
  File "c:\Users\domin\AppData\Local\Programs\Python\Python311\Lib\site-packages\click\core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\domin\AppData\Local\Programs\Python\Python311\Lib\site-packages\typer\core.py", line 778, in main
    return _main(
           ^^^^^^
  File "c:\Users\domin\AppData\Local\Programs\Python\Python311\Lib\site-packages\typer\core.py", line 216, in _main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "c:\Users\domin\AppData\Local\Programs\Python\